In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import pathlib
from sklearn.cluster import KMeans

import common.loss_utils as loss_utils

import sys
sys.path.append('utae-paps')
from src.backbones import utae_mod

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [4]:
FORECAST = 23
NUM_CLASSES = 81

model = utae_mod.UTAE(
    input_dim=1, # 10 for paper
    forecast_steps=FORECAST,
    encoder_widths=[32, 64, 64, 128],
    decoder_widths=[32, 32, 64, 128],
    out_conv=[32, NUM_CLASSES],
    str_conv_k=4,
    str_conv_s=2,
    str_conv_p=1,
    agg_mode="att_group",
    encoder_norm="group",
    n_head=16,
    d_model=256,
    d_k=4,
    encoder=False,
    return_maps=False,
    pad_value=None, # 0
    padding_mode="reflect",
)
model = model.to(DEVICE)

In [5]:
BATCH_SIZE = 1
x = torch.randn((BATCH_SIZE, 12, 1, 128, 128)).to(DEVICE)
y = torch.randint(low=0, high=NUM_CLASSES, size=(BATCH_SIZE, FORECAST, 64, 64)).long().to(DEVICE)

In [6]:
criterion = torch.nn.CrossEntropyLoss()


In [7]:
out = model(x)

STEP 0
STEP 1
STEP 2
STEP 3
STEP 4
STEP 5
STEP 6
STEP 7


RuntimeError: CUDA out of memory. Tried to allocate 76.00 MiB (GPU 0; 7.79 GiB total capacity; 6.42 GiB already allocated; 35.62 MiB free; 6.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
out.shape

torch.Size([1, 23, 81, 64, 64])

In [29]:
y.shape

torch.Size([1, 23, 64, 64])

In [30]:
b, t, k, w, h = out.shape
out_flat = out.reshape(b*t, k, w, h)
y_flat = y.reshape(b*t, w, h)


In [31]:
out_flat.shape

torch.Size([23, 81, 64, 64])

In [32]:
y_flat.shape

torch.Size([23, 64, 64])

In [33]:
loss = criterion(out_flat, y_flat)

In [34]:
loss

tensor(4.6216, device='cuda:0', grad_fn=<NllLoss2DBackward0>)

In [35]:
loss.backward()